# Обзор популярных методов обработки категориальных признаков

Текщая работа выполнена в рамках образовательного проекта "**Booster MDS**"

Формальным определением категориального признака можно считать утверждение, согластно которому под этим понятием подразумеваются все сущности описывающие объект и  не входящие в множество вещественных чисел. Бинарные признаки могут быть расценены в качестве частного случая категориальных, тем не менее корректность такой позиции является вопросом семантики и общеприянтой практикой является интерпретация таковых как отдельного типа признаков.     
Некоторые из методов машинного обучения являются чуствительными к характеру и типу представления признаков, описывающих обекты, а многие из них не умеют работать с категориальными признаками в виду своей сущности, предполагающую оперирование сугубо вещественными величинами.  
В рамках текущей работы я рассмотрю некоторые из способов обработки таких признаков и представления данных в необходимой форме.

In [6]:
import pandas as pd
import random

# В качесте примера будем использовать следующую серию, содержащую набор основных стихий
elements = ['Air', 'Fire', 'Water', 'Earth', 'DataScience']
df = pd.DataFrame({'cat': [elements[random.randint(0,4)] for _ in range(20)]})
df.head()

,cat
0,Water
1,Water
2,Earth
3,DataScience
4,Air


## Проецирование на числовую прямую (Label Encoder )

Суть данного подхода заключается в нумерации возможных значений (в случае если их конечное множество) и представлении какждого из них в качестве его порядкового номера

In [9]:
df.cat.replace({x: i for i, x in enumerate(elements)})

0     2
1     2
2     3
3     4
4     0
5     1
6     1
7     0
8     3
9     1
10    2
11    0
12    0
13    4
14    3
15    4
16    0
17    3
18    2
19    4
Name: cat, dtype: int64

Главным минусом данного подхода является появление определенного веса у того или иного значения признака за счет величины значения его представления, что может привести к возникновению ошибочных закономерностей

## Булеризация (One-Hot-Encoder)

Подход, при котором каждое значение искомого признака представляется в качестве нового, уже бинарного типа, содержащего  1 *(True)* для объекта которому соответствует  и 0 *(False)* в остальных случаях. Количество итоговых векторов равно изначальному количеству значений рассматриваемого признака.

In [10]:
pd.get_dummies(df)

,cat_Air,cat_DataScience,cat_Earth,cat_Fire,cat_Water
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,1,0,0
3,0,1,0,0,0
4,1,0,0,0,0
5,0,0,0,1,0
6,0,0,0,1,0
7,1,0,0,0,0
8,0,0,1,0,0
9,0,0,0,1,0


Этот метод позволяет сохранить всю информацию, но приводит к занчительному приросту в исходном датафрейме, что как следствие ведет к нагрузке на оперативную память и усложнению расчетов. Некоторые модели чувствительны к количеству признаков, и даже те, которые считатются устойчивыми - предполагают оптимальное соотношение количества признаков к количеству объектов.
Тем не менее, если число принимаемых признаком значений не велико - это один из лучших способов представления

## Частотное кодирование (Frequency Encoder)

Схоже в совей сущности с номинальным, только вместо порядкового номера подставляется частота появления значения

In [24]:
print(df.cat.value_counts())
{df.cat.value_counts()[i]: i for i in elements} 

Air            5
Water          4
DataScience    4
Earth          4
Fire           3
Name: cat, dtype: int64


{3: 'Fire', 4: 'DataScience', 5: 'Air'}

Я решил оставить текущий пример и не синтезировать новый, подходящий под текекущий подход, потому как данная ситуация отлично показывает главную проблему: типичный случай, когда частоты у некоторых значений совпадают, также теряется часть информации и возникают ложные связи. Текущий подход опрадан только если количество значений велико, а сохранение минимальной размерности датафрейма критично.

## Группа целевого кодирования

К этой группе относится целый ряд подходов, клюевой особенностью которых является представление значения признака через целевую переменную путем определенных преобразований или интерпетаций. это может быть использование среднего значения или пробразование с помощью одной из формул. Главным преимуществом таких подходов является укрепление связи между текзим признаком и таргетом, что позитивно сказывается на точности модели. К ним относятся TE, M-Estimate Encoder, James-Stein Encoder, Саша ты и сам знаешь откуда я это прочитал